In [ ]:
import numpy as np
import os
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras import regularizers, Input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [32]:
data_dir = ('./data/')
class_names = ["airplane", "apple", "bat", "cat", "circle", "clock", "cloud",
               "crown", "diamond", "dog", "donut", "face", "fish", "hexagon",
                "hot_dog", "lightning", "mountain", "river", "skull",
                "smiley_face", "square", "star", "sun", "t-shirt", "tree"]

In [ ]:
X = []
y = []

In [34]:
for label, class_name in enumerate(class_names):
    # Create the path to the .npy file (e.g., './data/apple.npy')
    file_path = os.path.join(data_dir, f"{class_name}.npy")

    # Load the .npy file → shape: (2000, 28, 28)
    data = np.load(file_path)

    X.append(data)

    # Add labels to y → [label, label, label, ..., label] (length = number of samples)
    y.append(np.full((data.shape[0]), label))


In [35]:
# Stack all image arrays vertically → final shape: (total_samples, 28, 28)
X = np.vstack(X)

# Stack all label arrays horizontally → final shape: (total_samples,)
y = np.hstack(y)

In [ ]:
X = StandardScaler().fit_transform(X)

### Max Normalization

In [ ]:
# # Normalize pixel values to range 0–1 (from 0–255)
# X = X.astype('float32') / 255.0
# X.shape

(750000, 784)

In [ ]:
# Flatten each image from 28x28 → 784 for dense layers
# X = X.reshape(X.shape[0], -1)  # shape becomes (total_samples, 784)

In [38]:
X_train, X_, y_train, y_ = train_test_split(X,y, test_size=0.2, random_state=42, stratify=y)
# `stratify=y` ensures equal class distribution in both training and validation sets.

X_cv, X_test, y_cv, y_test = train_test_split(X_,y_, test_size=0.5, random_state=42, stratify=y_)

In [ ]:
model = Sequential([
    Input(shape=(784,), name="input_vector"),
    Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    Dropout(0.4)
    Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    Dropout(0.3),
    Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    Dropout(0.2),
    Dense(len(class_names), activation='linear')
])

In [ ]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy']
)

In [ ]:
# Train
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ReduceLROnPlateau(patience=3, factor=0.5)
]

In [41]:
model.fit(X_train, y_train, validation_data=(X_cv, y_cv), epochs=50, callbacks=callbacks)

Epoch 1/40
18750/18750 ━━━━━━━━━━━━━━━━━━━━ 58s 3ms/step - loss: 1.6175
Epoch 2/40
18750/18750 ━━━━━━━━━━━━━━━━━━━━ 56s 3ms/step - loss: 1.2957
Epoch 3/40
18750/18750 ━━━━━━━━━━━━━━━━━━━━ 89s 3ms/step - loss: 1.2768
Epoch 4/40
18750/18750 ━━━━━━━━━━━━━━━━━━━━ 61s 3ms/step - loss: 1.2697
Epoch 5/40
18750/18750 ━━━━━━━━━━━━━━━━━━━━ 59s 3ms/step - loss: 1.2611
Epoch 6/40
18750/18750 ━━━━━━━━━━━━━━━━━━━━ 55s 3ms/step - loss: 1.2562
Epoch 7/40
18750/18750 ━━━━━━━━━━━━━━━━━━━━ 56s 3ms/step - loss: 1.2539
Epoch 8/40
18750/18750 ━━━━━━━━━━━━━━━━━━━━ 56s 3ms/step - loss: 1.2513
Epoch 9/40
18750/18750 ━━━━━━━━━━━━━━━━━━━━ 57s 3ms/step - loss: 1.2507
Epoch 10/40
18750/18750 ━━━━━━━━━━━━━━━━━━━━ 57s 3ms/step - loss: 1.2476
Epoch 11/40
18750/18750 ━━━━━━━━━━━━━━━━━━━━ 58s 3ms/step - loss: 1.2474
Epoch 12/40
18750/18750 ━━━━━━━━━━━━━━━━━━━━ 59s 3ms/step - loss: 1.2479
Epoch 13/40
18750/18750 ━━━━━━━━━━━━━━━━━━━━ 56s 3ms/step - loss: 1.2480
Epoch 14/40
18750/18750 ━━━━━━━━━━━━━━━━━━━━ 58s 3ms/step - 

In [42]:
y_cv_pred_probs = model.predict(X_cv)   #predict probability for each label
y_cv_pred = np.argmax(y_cv_pred_probs, axis=1)  # selects the label with maximum probability

2344/2344 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step


In [43]:
print(accuracy_score(y_cv,y_cv_pred))

0.7578266666666666


In [44]:
y_test_pred_probs = model.predict(X_test) 
y_test_pred = np.argmax(y_test_pred_probs, axis=1)
print(accuracy_score(y_test,y_test_pred))

2344/2344 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
0.75512


In [45]:
model.save("doodle_model.keras")